In [4]:
import pdfplumber
import re
import json
import os
import PyPDF2
import re
import pandas as pd
import sys
import os


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph


In [7]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")


NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, enhanced_schema=True)

print(graph.schema)

Node properties:
- **Ordine**
  - `nome`: STRING Available options: ['Ordine della Galassia di Andromeda', 'Ordine dei Naturalisti', 'Ordine degli Armonisti']
  - `restrizioni`: STRING Available options: ['privo di lattosio']
  - `obiettivo`: STRING Available options: ['creare sapori complessi senza ingredienti lattiero', 'custodire la purezza degli ingredienti', 'creare piatti che parlano al cuore']
  - `cucina`: STRING Available options: ['diplomazia culinaria intergalattica', 'esaltare la semplicità con maestria', 'sinfonia gastronomica']
  - `ingredienti vietati`: LIST Min Size: 3, Max Size: 3
  - `caratteristica fisiologica`: STRING Available options: ['Intolleranza al lattosio']
  - `trasformazioni`: STRING Available options: ['nessuna trasformazione drastica, niente manipolazi']
  - `principio`: STRING Available options: ['Principio di Non Alterazione']
  - `filosofia`: STRING Available options: ['Preservare la perfezione innata degli ingredienti', 'Il cibo deve entrare in rison

In [14]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)
llm.invoke("Ciao, come posso aiutarti?")

AIMessage(content='Ciao! Grazie per esserti offerto di aiutarmi. Avrei bisogno di assistenza con [specifica richiesta]. Potresti darmi una mano?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1133, 'prompt_tokens': 15, 'total_tokens': 1148, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 1088, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_f6ff3bb326', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {

In [9]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

In [10]:
chain.invoke({"query": "Who played in Top Gun?"})





> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n) RETURN n LIMIT 0

Full Context:
[]

> Finished chain.


{'query': 'Who played in Top Gun?', 'result': "I don't know the answer."}